# Look at instances of top terms in corpora

In [1]:
# Load corpora/splits
import pickle
import os

data_dirpath = '/home/mamille3/hegemonic_hate/data'
# splits_fpath = os.path.join(data_dirpath, 'combined_identity_splits_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')
split_type = 'identities'
splits_fpath = os.path.join(data_dirpath, f'combined_{split_type}_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')
with open(splits_fpath, 'rb') as f:
    corpora = pickle.load(f)
    
corpora.keys()

dict_keys([('latinx people',), ('jews',), ('women',), ('men',), ('white people',), ('asian people',), ('black people',), ('christians',), ('lgbtq+ people',), ('muslims and arabic/middle eastern people',)])

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

split = ('lgbtq+ people',)
term = 'bathroom'
data = corpora[split]['all'].query('hate').copy()
data['text_lower'] = data.text.str.lower()
selected = data[data.text_lower.str.contains(r'\b{}\b'.format(term))]
selected.sample(min(len(selected), 20)).loc[:, ['text_lower', 'hate', 'target_groups', 'dataset', 'targetMinority']]

,text_lower,hate,target_groups,dataset,targetMinority
text_id,,,,,
5663983,"quite frankly, it is hard to imagine how transgenders can be accommodated in the military. so the guy wants to be a girl. how do the real girls feel when he is put in their barracks and uses their bathroom. those who need special accommodations do not belong in the military.",True,"[men, women, transgender people]",civilcomments,NaN
329621,"""i am not saying that transgender people are predators. not by a long shot. what i am saying is that there are countless deviant men in this world who will pretend to be transgender as a means of gaining access to the people they want to exploit, namely women and children."" \n\n""there’s no way to make everyone happy in the situation of transgender locker room use. so the priority ought to be finding a way to keep everyone safe. i’d much rather risk hurting a smaller number of people’s feelings by asking transgender people to use a single-occupancy restroom that still offers safety than risk jeopardizing the safety of thousands of women and kids with a policy that gives would-be predators a free pass.""\n\n http://thefederalist.com/2015/11/23/a-rape-survivor-speaks-out-about-transgender-bathrooms/\n\nso far, i haven't heard of many transgenders who id as men pushing for bathroom rights. have you?",True,"[men, women, transgender people]",civilcomments,NaN
5476941,"so dishonest! where are all the bathroom, locker room crimes against transpeople...please give reports!!! \n.\na transperson could ask the same, please give reports of all the transpeople who have committed locker room sexual crimes against women and children?\n.\nnew york and california have had laws on the books for years, new york for 10. could you supply statistics of increased sexual crimes committed in these states by lgbtq peoples. this should prove or disprove your point.",True,"[women, transgender people, gay and lesbian people]",civilcomments,NaN
1032243,"if it's a ""decency issue"" then you can count yourself out, as can trump and all of his supporters. none of you has an ounce of decency. for shame.\n\n i'll take a trans person in the bathroom any day over trump. ""common sense"" says trump is a pathological liar, sexual predator and cheat, among other things. i wouldn't want him anywhere near me or my child, especially in a restroom.",True,[transgender people],civilcomments,NaN
47221,+j m kept them out of the female bathroom while obama let every chester molester into the female bathroom and made a law about it as soon as trump took office that law was removed women and girls now have their privacy back in a public bathroom without having to worry about some fucking pervert transgender fucking weirdo coming in there,True,"[non-binary people, transgender men, transgender people, transgender women, women, bisexual people, gay people, lesbian women, straight people]",kennedy2020,NaN
4850,btw fuckers and all shitposters here on gab i could never ever be gay because it hurts like a motherfucker when i use the bathroom to take a shit and end up with damn hemorrhoids for about a week hurts like a bastard face,True,[gay men],sbic,"[""gay men""]"
7182051,"you can totally tell that the person that wrote this was incredibly trans phobic. ""i don't want to be trans phobic ."" then continues to perpetuate the idea that trans people will start raping people or something if they're given access to the bathroom. you aren't being subtle sweetheart, just because you say you aren't doesn't mean your wording doesn't say you are. \n if you're going to try to be an a55hole and a nice person at the same time, you're doing it wrong.",True,[transgender people],civilcomments,NaN
7075643,"i'm sure if ike had told the guys storming normandy that they were going into harms way to make gay marriage legal, put trannies in the girls bathroom and tear down statues of george washington, they would have attacked washington dc instead of the nazis.",True,"[transgender people, g

# Prepare corpora for SAGE
Then run it at ~/SAGE/py-sage (repo fork at https://github.com/michaelmilleryoder/SAGE/tree/master/py-sage)  
Take a look at runSage_grid.py, which runs runSage.py over multiple parameter settings

In [12]:
# Load corpora/splits
import pickle
import os
import pandas as pd
from tqdm.notebook import tqdm
import spacy
import re

nlp = spacy.load('en_core_web_sm', exclude=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner'])
nlp.add_pipe('sentencizer')

data_dirpath = '/home/mamille3/hegemonic_hate/data'
# splits_fpath = os.path.join(data_dirpath, 'combined_identity_splits_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')

split_types = ['identities', 'categories', 'power']
for split_type in split_types:
    splits_fpath = os.path.join(data_dirpath, f'combined_{split_type}_kennedy2020+sbic+hatexplain+civilcomments_0.3hate.pkl')
    with open(splits_fpath, 'rb') as f:
        corpora = pickle.load(f)

    texts = {}

    split_transform = {
        ('race/ethnicity',): 'race/ethnicity',
        ('religion',): 'religion',
        ('gender', 'sexuality'): 'gender/sexuality',
        ('hegemonic',): 'hegemonic',
        ('marginalized',): 'marginalized',
    }

    # for split in list(corpora.keys())[:1]:
    for split in tqdm(corpora):
        data = pd.concat([corpora[split]['train'], corpora[split]['test']])
        hate = data.query('hate')
        texts[split] = hate.text.tolist()

        # Process, save out data
        processed = []
        inp = texts[split]
        # for doc in tqdm(nlp.pipe(inp), total=len(inp)):
        for doc in nlp.pipe(inp):
            sents = [' '.join([tok.text for tok in sent]).strip().lower() for sent in doc.sents]
            processed.extend(sents)

        # Save out
        out_dirpath = os.path.join(data_dirpath, split_type)
        if not os.path.exists(out_dirpath):
            os.mkdir(out_dirpath)
        outpath = os.path.join(out_dirpath, f'{re.sub(r"[ /,]", "_", split_transform.get(split, split[0]))}_sents.txt')
        with open(outpath, 'w') as f:
            for sent in processed:
                f.write(f'{sent}\n')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

# Read, format SAGE output

In [13]:
import os
import pandas as pd
from IPython.display import display

split_types = ['identities', 'categories', 'power']
vocab_sizes = [3000] # [1000, 1500, 2000, 3000, 5000, 10000]
smoothing_rates = [10, 20, 50, 100]
for vocab_size in vocab_sizes:
    print(f'{vocab_size} vocab')
    for smoothing_rate in smoothing_rates:
        print(f'{smoothing_rate} smoothing')
        for split_type in split_types:
            n_words = 10
            sage_outpath = f'/home/mamille3/SAGE/py-sage/output/{split_type}_{n_words}words_{vocab_size}vocab_{smoothing_rate}smoothing.csv'
            results = pd.read_csv(sage_outpath, sep='\t')
            # results['split'] = results.source.str.slice(len(f'input/{split_type}/'), -1*(len('_sents.txt')))
            results['split'] = results.source.map(lambda x: os.path.basename(x)[:-1*(len('_sents.txt'))])
            # results

            pd.set_option('display.max_colwidth', None)
            top_terms = results.groupby('split').agg({'word': ', '.join})
            if split_type == 'identity':
                top_terms.drop('people_of_color', inplace=True)
                top_terms.index = top_terms.index.str.replace('_', ' ')
                top_terms.index = top_terms.index.str.capitalize()
            elif split_type == 'categories':
                top_terms.index = top_terms.index.str.replace('_', '/')
            top_terms.index.name = split_type.capitalize()
            top_terms.columns = ['Top terms']
            if split_type == 'identity':
                top_terms = top_terms.rename(index={'Lgbtq+ people': 'LGBTQ+ people', 'Muslims and arabic middle eastern people': 'Muslims, Arabs'})
            display(top_terms)

3000 vocab
10 smoothing


,Top terms
Identities,
asian_people,"ching, chinese, chong, chinaman, china, asian, japanese, chink, asians, korean"
black_people,"ethiopian, niggas, black, nigga, blm, negro, africa, paint, nfl, nigger"
christians,"priest, catholic, priests, jesus, bible, christianity, catholics, christ, francis, church"
jews,"jewish, chamber, holocaust, oven, gas, zionists, jews, hitler, zionist, jew"
latinx_people,"beaner, latino, latinos, mexico, latin, mexican, spic, mexicans, beaners, puerto"
lgbtq+_people,"transgender, transgendered, transgenderism, trans, transgenders, bisexual, bathrooms, disorder, queers, bathroom"
men,"divorce, dudes, negative, movies, soy, professional, field, bloom, male, men"
muslims_and_arabic_middle_eastern_people,"allahu, akbar, muzrat, islam, islamic, mohammed, muslim, islamophobia, islamist, isis"
white_people,"redneck, supremacists, supremacist, supremacy, white, trailer, shootings, whitey, mudshark, fascist"


,Top terms
Categories,
gender/sexuality,"dyke, hoes, transgender, feminine, trans, dykes, fag, queer, lesbian, locker"
race/ethnicity,"ethiopian, chinese, asian, mexicans, black, hispanic, blacks, asians, nationalists, races"
religion,"priest, catholic, catholics, christians, evangelicals, rabbi, christianity, christian, clergy, bishops"


,Top terms
Power,
hegemonic,"priest, priests, catholic, jesus, allcaps, catholics, virgin, bloom, dudes, pope"
marginalized,"muslim, muslims, she, islam, woman, her, nigger, black, jews, women"


20 smoothing


...

,Top terms
Power,
hegemonic,"priest, catholic, priests, jesus, allcaps, catholics, virgin, church, devil, dress"
marginalized,"muslim, muslims, she, islam, her, woman, nigger, black, jews, women"


100 smoothing


,Top terms
Identities,
asian_people,"chinese, asian, china, asians, japanese, ching, chong, chink, chinaman, indian"
black_people,"black, niggas, nigga, nigger, africa, blacks, blm, african, negro, lives"
christians,"priest, catholic, jesus, christians, christian, church, bible, christianity, priests, catholics"
jews,"jewish, jews, jew, holocaust, israel, hitler, kike, gas, zionist, oven"
latinx_people,"mexican, mexicans, latinos, latino, mexico, spic, hispanic, beaner, hispanics, latin"
lgbtq+_people,"transgender, trans, transgendered, transgenders, gay, faggot, faggots, bisexual, queers, fag"
men,"men, male, man, divorce, negative, dudes, boys, priests, dad, movies"
muslims_and_arabic_middle_eastern_people,"muslim, islam, islamic, muslims, terrorist, isis, terrorists, bomb, iran, radical"
white_people,"redneck, supremacists, white, supremacist, trash, mudshark, whites, supremacy, nazi, nazis"


,Top terms
Categories,
gender/sexuality,"hoes, faggot, dyke, woman, transgender, fag, sexual, sexist, women, female"
race/ethnicity,"black, blacks, chinese, whites, asian, asians, white, racism, africa, racist"
religion,"catholic, christians, christian, priest, catholics, christianity, religion, church, islam, jesus"


,Top terms
Power,
hegemonic,"priest, catholic, priests, jesus, allcaps, catholics, church, white, supremacists, virgin"
marginalized,"muslim, muslims, she, black, her, women, woman, nigger, in, to"


In [21]:
# Just look at an individual run's output
# Format for LaTeX

import os
import pandas as pd
from IPython.display import display

split_types = ['identities', 'categories', 'power']
vocab_size = 3000
n_words = 10
smoothing = 50
for split_type in split_types:
    sage_outpath = f'/home/mamille3/SAGE/py-sage/output/{split_type}_{n_words}words_{vocab_size}vocab_{smoothing}smoothing.csv'
    results = pd.read_csv(sage_outpath, sep='\t')
    # results['split'] = results.source.str.slice(len(f'input/{split_type}/'), -1*(len('_sents.txt')))
    results['split'] = results.source.map(lambda x: os.path.basename(x)[:-1*(len('_sents.txt'))])
    # results

    pd.set_option('display.max_colwidth', None)
    top_terms = results.groupby('split').agg({'word': ', '.join})
    if split_type == 'identities':
        if 'people of color' in top_terms.index:
            top_terms.drop('people_of_color', inplace=True)
        top_terms.index = top_terms.index.str.replace('_', ' ').str.replace(' people', '')
        top_terms.index = top_terms.index.str.capitalize()
    elif split_type == 'categories':
        top_terms.index = top_terms.index.str.replace('_', '/')
    top_terms.index.name = split_type.capitalize()
    top_terms.columns = ['Top terms']
    if split_type == 'identities':
        top_terms = top_terms.rename(index={'Lgbtq+': 'LGBTQ+', 'Muslims and arabic middle eastern': 'Muslims, Arabs'})
    display(top_terms)

    latex = top_terms.style.to_latex(hrules=True, label=f'{split_type}_sage', caption=f'Most representative terms in corpora divided by target identity {split_type} from SAGE', environment='table*')
    latex = latex.replace(' & Top terms', f'{split_type.capitalize()} & Top terms').replace(f'{split_type.capitalize()} &  \\\\\n', '').replace(' people', '').replace('hegemonic', 'dominant')
    print(latex)

,Top terms
Identities,
Asian,"chinese, china, asian, ching, chong, asians, japanese, chinaman, chink, japan"
Black,"niggas, black, nigga, nigger, africa, blm, negro, ethiopian, blacks, african"
Christians,"priest, catholic, jesus, priests, bible, christians, christianity, christian, church, catholics"
Jews,"jewish, jews, holocaust, jew, israel, hitler, gas, oven, zionist, kike"
Latinx,"latinos, latino, mexico, mexican, mexicans, beaner, spic, latin, hispanic, beaners"
LGBTQ+,"transgender, transgendered, trans, transgenders, bisexual, queers, bathroom, bathrooms, fag, gay"
Men,"divorce, dudes, men, male, negative, movies, man, priests, soy, dad"
"Muslims, Arabs","islam, muslim, islamic, muslims, isis, terrorist, terrorists, iran, bomb, radical"
White,"redneck, supremacists, white, supremacist, supremacy, mudshark, trash, fascist, whites, shootings"


\begin{table*}
\caption{Most representative terms in corpora divided by target identity identities from SAGE}
\label{identities_sage}
\begin{tabular}{ll}
\toprule
Identities & Top terms \\
\midrule
Asian & chinese, china, asian, ching, chong, asians, japanese, chinaman, chink, japan \\
Black & niggas, black, nigga, nigger, africa, blm, negro, ethiopian, blacks, african \\
Christians & priest, catholic, jesus, priests, bible, christians, christianity, christian, church, catholics \\
Jews & jewish, jews, holocaust, jew, israel, hitler, gas, oven, zionist, kike \\
Latinx & latinos, latino, mexico, mexican, mexicans, beaner, spic, latin, hispanic, beaners \\
LGBTQ+ & transgender, transgendered, trans, transgenders, bisexual, queers, bathroom, bathrooms, fag, gay \\
Men & divorce, dudes, men, male, negative, movies, man, priests, soy, dad \\
Muslims, Arabs & islam, muslim, islamic, muslims, isis, terrorist, terrorists, iran, bomb, radical \\
White & redneck, supremacists, white, supremacist

,Top terms
Categories,
gender/sexuality,"hoes, dyke, transgender, faggot, fag, sexist, sexual, lesbian, hoe, dykes"
race/ethnicity,"chinese, black, blacks, asian, asians, mexicans, whites, africa, supremacists, supremacist"
religion,"catholic, priest, catholics, christians, christian, christianity, religion, church, jesus, koran"


\begin{table*}
\caption{Most representative terms in corpora divided by target identity categories from SAGE}
\label{categories_sage}
\begin{tabular}{ll}
\toprule
Categories & Top terms \\
\midrule
gender/sexuality & hoes, dyke, transgender, faggot, fag, sexist, sexual, lesbian, hoe, dykes \\
race/ethnicity & chinese, black, blacks, asian, asians, mexicans, whites, africa, supremacists, supremacist \\
religion & catholic, priest, catholics, christians, christian, christianity, religion, church, jesus, koran \\
\bottomrule
\end{tabular}
\end{table*}



,Top terms
Power,
hegemonic,"priest, catholic, priests, jesus, allcaps, catholics, virgin, church, devil, dress"
marginalized,"muslim, muslims, she, islam, her, woman, nigger, black, jews, women"


\begin{table*}
\caption{Most representative terms in corpora divided by target identity power from SAGE}
\label{power_sage}
\begin{tabular}{ll}
\toprule
Power & Top terms \\
\midrule
dominant & priest, catholic, priests, jesus, allcaps, catholics, virgin, church, devil, dress \\
marginalized & muslim, muslims, she, islam, her, woman, nigger, black, jews, women \\
\bottomrule
\end{tabular}
\end{table*}



# Run SAGE with different settings
Tried to run it directly from Python script instead of through a shell command, but it's Python 2.7

In [1]:
import os
from tqdm.notebook import tqdm
os.chdir('/home/mamille3/SAGE/py-sage/')
from runSage import runSage

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(s.its,'/',s.max_its,change)? (deltaIterator.py, line 16)

In [ ]:
vocab_settings = [1500, 3000, 5000]
for vocab_setting in tqdm(vocab_settings):
    cmd = 